In [2]:
import tensorflow as tf
from tensorflow import feature_column as fc
import shared_embedding_column as sec
tf.compat.v1.disable_eager_execution()

# 测试API

In [3]:
feed_col = fc.categorical_column_with_hash_bucket('featureField57', 7500000, tf.int64)
long_pref_feed_col = fc.categorical_column_with_hash_bucket('featureField58', 7500000, tf.int64)
short_pref_feed_col = fc.categorical_column_with_hash_bucket('featureField59', 7500000, tf.int64)
feed_col,long_pref_feed_col,short_pref_feed_col

(HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64),
 HashedCategoricalColumn(key='featureField58', hash_bucket_size=7500000, dtype=tf.int64),
 HashedCategoricalColumn(key='featureField59', hash_bucket_size=7500000, dtype=tf.int64))

In [4]:
shared_list = sec.shared_embedding_columns_v3(categorical_columns=[feed_col, long_pref_feed_col, short_pref_feed_col], dimension=128, combiner='sqrtn', initializer=tf.keras.initializers.VarianceScaling(distribution='uniform'))
shared_list

[SharedEmbeddingColumn(base_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), categorical_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), shared_embedding_column_creator=<shared_embedding_column.SharedEmbeddingColumnCreator object at 0x7ff0647e2a50>, combiner='sqrtn', max_norm=None, use_safe_embedding_lookup=True),
 SharedEmbeddingColumn(base_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), categorical_column=HashedCategoricalColumn(key='featureField58', hash_bucket_size=7500000, dtype=tf.int64), shared_embedding_column_creator=<shared_embedding_column.SharedEmbeddingColumnCreator object at 0x7ff0647e2a50>, combiner='sqrtn', max_norm=None, use_safe_embedding_lookup=True),
 SharedEmbeddingColumn(base_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), categorical_column=HashedCategoricalColumn(key='featur

# 测试ShareEmbeddingColumn

## 创建SharedEmbeddingColumnCreator的List

In [5]:
categorical_columns = [feed_col, long_pref_feed_col, short_pref_feed_col]

In [6]:
sorted_columns = sorted(categorical_columns, key=lambda x: x.name)
sorted_columns

[HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64),
 HashedCategoricalColumn(key='featureField58', hash_bucket_size=7500000, dtype=tf.int64),
 HashedCategoricalColumn(key='featureField59', hash_bucket_size=7500000, dtype=tf.int64)]

In [7]:
shared_embedding_collection_name = '_'.join(c.name for c in sorted_columns)
shared_embedding_collection_name += '_shared_embedding'
shared_embedding_collection_name

'featureField57_featureField58_featureField59_shared_embedding'

In [8]:
column_creator =sec.SharedEmbeddingColumnCreator(dimension=128, initializer=tf.keras.initializers.VarianceScaling(distribution='uniform'),
                                                  ckpt_to_load_from=None, tensor_name_in_ckpt=None,
                                                  num_buckets=7500000, base_column=categorical_columns[0], trainable=True,
                                                  name=shared_embedding_collection_name,
                                                  use_safe_embedding_lookup=True)
column_creator

In [9]:
result = []
for column in categorical_columns:
    result.append(column_creator(categorical_column=column, combiner='sqrtn', max_norm=None))
result

[SharedEmbeddingColumn(base_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), categorical_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), shared_embedding_column_creator=<shared_embedding_column.SharedEmbeddingColumnCreator object at 0x7ff0647bc310>, combiner='sqrtn', max_norm=None, use_safe_embedding_lookup=True),
 SharedEmbeddingColumn(base_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), categorical_column=HashedCategoricalColumn(key='featureField58', hash_bucket_size=7500000, dtype=tf.int64), shared_embedding_column_creator=<shared_embedding_column.SharedEmbeddingColumnCreator object at 0x7ff0647bc310>, combiner='sqrtn', max_norm=None, use_safe_embedding_lookup=True),
 SharedEmbeddingColumn(base_column=HashedCategoricalColumn(key='featureField57', hash_bucket_size=7500000, dtype=tf.int64), categorical_column=HashedCategoricalColumn(key='featur

### __call__
创建对应featureField的SharedEmbeddingColumn(name='featureField57_shared_embedding')

### 创建shared的embedding_weights

In [10]:
def get_or_create_embedding_weights(self, base_column, state_manager):
    """
    0. 关键是SharedEmbeddingColumnCreator的创建embedding_weights的方法；
    1. 取默认图的key，作为创建share_embedding_weights的标识，如果已有则不再创建；
    2. name='featureField57_featureField58_featureField59_shared_embedding'。
    
    """
    key = ops.get_default_graph()._graph_key  # pylint: disable=protected-access
    if key not in self._embedding_weights:
      embedding_shape = (self._num_buckets, self._dimension)
      var = state_manager.create_variable(
        base_column,
        name=self._name,
        shape=embedding_shape,
        dtype=dtypes.float32,
        trainable=self._trainable,
        use_resource=True,
        initializer=self._initializer)
      if self._ckpt_to_load_from is not None:
        to_restore = var
        if isinstance(to_restore, variables.PartitionedVariable):
          to_restore = to_restore._get_variable_list()  # pylint: disable=protected-access
        checkpoint_utils.init_from_checkpoint(
            self._ckpt_to_load_from, {self._tensor_name_in_ckpt: to_restore})
      self._embedding_weights[key] = var
    else:
      pass
    return self._embedding_weights[key]

## SharedEmbeddingColumn

### 重写create_state

In [11]:
def create_state(self, state_manager):
    """
    0. 关键是创建变量这，替换成了shared_embedding_column_creator的创建embedding_weights的方法，因此会共用一个creator；
    1. 一般调用state_manager.create_variable，创建weights；
    2. 此处，调用sec_creator的创建embedding_weights方法，共用一套ID体系的，不再重复创建
    3. self.shared_embedding_column_creator.get_or_create_embedding_weights(self.base_column, state_manager)
    """
    pass

### 获取SparseTensor查找shared_embedding_weights

In [12]:
def _get_dense_tensor_internal(self, transformation_cache, state_manager):
    """Private method that follows the signature of _get_dense_tensor."""
    # This method is called from a variable_scope with name _var_scope_name,
    # which is shared among all shared embeddings. Open a name_scope here, so
    # that the ops for different columns have distinct names.
    with ops.name_scope(None, default_name=self.name):
        # 获取id，weight的SparseTensor
        sparse_tensors = self.categorical_column.get_sparse_tensors(transformation_cache, state_manager)
        sparse_ids = sparse_tensors.id_tensor
        sparse_weights = sparse_tensors.weight_tensor
        # 获取这类ID对应的 embedding_weights
        embedding_weights = state_manager.get_variable(self.base_column, name=self.shared_embedding_column_creator.name)
        # 对id和weights的SparseTensor，进行embedding_lookup
        sparse_id_rank = tensor_shape.dimension_value(sparse_ids.dense_shape.get_shape()[0])
        embedding_lookup_sparse = embedding_ops.safe_embedding_lookup_sparse
        if (not self.use_safe_embedding_lookup and sparse_id_rank is not None and sparse_id_rank <= 2):
            embedding_lookup_sparse = embedding_ops.embedding_lookup_sparse_v2
        # Return embedding lookup result.
        return embedding_lookup_sparse(
            embedding_weights,sparse_ids,sparse_weights,combiner=self.combiner,
            name='%s_weights' % self.name,max_norm=self.max_norm)

# 结论
1. 中间利用SharedEmbeddingColumnCreator，创建SharedEmbeddingColumn；
2. 创建embedding_weights，无则创建，有则获取；
3. 对SharedEmbeddingColumn，修改create_state方法，使用creator的get_or_create_embedding_weights实现共用一套embedding_weights